# Particle Filter Implementation with TensorFlow and TensorFlow Probability

## Model Definition
**Dynamic/State Equation:**
```
X(k+T) = AX(k) + GW(k)
```

**Observation Equation:**
```
Y(k) = [sqrt(X(k)[0]^2 + X(k)[1]^2);  # Range
        atan2(X(k)[1], X(k)[0])]  + V(k)  # Bearing
```

**Parameters:**
- State: X = [pos_x, pos_y, vel_x, vel_y]
- Process noise: W ~ N(0, Q)
- Observation noise: V ~ N(0, R)

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt

tfd = tfp.distributions

# Set random seed for reproducibility
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"TensorFlow Probability version: {tfp.__version__}")

In [ ]:
# Load observation data - exactly as in Julia code
# T = collect(5:5:200)
T_values = tf.constant([5., 10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60., 65., 70., 75., 80., 
                        85., 90., 95., 100., 105., 110., 115., 120., 125., 130., 135., 140., 145., 150., 
                        155., 160., 165., 170., 175., 180., 185., 190., 195., 200.], dtype=tf.float64)

Range = tf.constant([
    27.942258, 27.963234, 27.204243, 26.540936, 26.135477, 24.921334, 26.008057, 23.451562,
    23.475078, 22.859580, 22.441457, 20.646783, 21.282532, 21.026488, 21.922655, 23.111669,
    24.607487, 26.674349, 27.889730, 29.995459, 31.511762, 31.999494, 32.519595, 33.830023,
    33.801987, 34.345191, 34.878209, 35.858721, 37.624024, 39.121417, 40.449665, 40.245695,
    40.242868, 41.246978, 41.759315, 41.612828, 42.370687, 43.895030, 45.125174, 48.508522
], dtype=tf.float64)

Theta = tf.constant([
    0.817212, 0.844120, 0.870304, 0.939488, 0.977246, 1.009955, 1.095079, 1.123509,
    1.195098, 1.317959, 1.404055, 1.569062, 1.679964, 1.751688, 1.908451, 2.032702,
    2.144561, 2.159674, 2.172540, 2.231149, 2.217351, 2.157791, 2.140403, 2.165825,
    2.121799, 2.082291, 2.126691, 2.102974, 2.034597, 2.031375, 2.024936, 2.053166,
    2.010958, 1.983078, 2.000593, 2.042380, 2.023929, 2.044431, 2.006127, 1.991178
], dtype=tf.float64)

# Use the first timestep value as T (in Julia: df.T[1])
T_timestep = T_values[0]

# Y_observation = hcat(df.Range, df.Theta) in Julia
Y_observations = tf.stack([Range, Theta], axis=1)
num_steps = tf.shape(Y_observations)[0].numpy()

print(f"Loaded {num_steps} observations")
print(f"Time step T: {T_timestep.numpy()}")
print(f"Observations shape: {Y_observations.shape}")
print(f"First observation: Range={Range[0].numpy():.6f}, Theta={Theta[0].numpy():.6f}")

In [ ]:
# Model parameters - exactly as in Julia code
T = T_timestep

# State transition matrix: A = [1 0 T 0; 0 1 0 T; 0 0 1 0; 0 0 0 1]
A = tf.constant([
    [1., 0., T, 0.],
    [0., 1., 0., T],
    [0., 0., 1., 0.],
    [0., 0., 0., 1.]
], dtype=tf.float64)

# Process noise matrix: G = [0 0; 0 0; T 0; 0 T]
G = tf.constant([
    [0., 0.],
    [0., 0.],
    [T, 0.],
    [0., T]
], dtype=tf.float64)

# Process noise covariance: Q = [0.001 0; 0 0.001]
Q = tf.constant([
    [0.001, 0.],
    [0., 0.001]
], dtype=tf.float64)

# Observation noise covariance: R = [0.25 0; 0 5*10^(-4)]
R = tf.constant([
    [0.25, 0.],
    [0., 5e-4]
], dtype=tf.float64)

# Create distributions
state_noise_dist = tfd.MultivariateNormalFullCovariance(
    loc=tf.zeros(2, dtype=tf.float64),
    covariance_matrix=Q
)

print("Model parameters initialized")
print(f"A shape: {A.shape}, G shape: {G.shape}")
print(f"Q shape: {Q.shape}, R shape: {R.shape}")

In [ ]:
# Define functions - matching Julia implementation

def state_transition(X, W):
    """State transition function: X(k+1) = A*X(k) + G*W(k)
    Julia: fcn_X_pre(X, W) = A * X + G * W
    
    Args:
        X: State tensor of shape (N, 4)
        W: Process noise tensor of shape (N, 2)
    
    Returns:
        New state tensor of shape (N, 4)
    """
    # For each particle: X_new = A @ X + G @ W
    # Vectorized: (N,4) @ (4,4)^T + (N,2) @ (4,2)^T = (N,4) + (N,4)
    return tf.matmul(X, A, transpose_b=True) + tf.matmul(W, G, transpose_b=True)

def observation_function(X):
    """Compute expected observation from state
    Julia: [abs.(X[1] + X[2] * im); atan(X[2], X[1])]
    Note: Julia uses 1-indexing, so X[1]=pos_x, X[2]=pos_y
    
    Args:
        X: State tensor of shape (N, 4) where X[:,0]=pos_x, X[:,1]=pos_y
    
    Returns:
        Observation tensor of shape (N, 2) - [range, bearing]
    """
    # Range: |pos_x + i*pos_y| = sqrt(pos_x^2 + pos_y^2)
    range_pred = tf.sqrt(X[:, 0]**2 + X[:, 1]**2)
    # Bearing: atan2(pos_y, pos_x)
    bearing_pred = tf.atan2(X[:, 1], X[:, 0])
    return tf.stack([range_pred, bearing_pred], axis=1)

def observation_residual(X, Y):
    """Compute observation residual V = Y - h(X)
    Julia: fcn_V_between_pre_and_ob(X, Y) = Y - [abs.(X[1] + X[2] * im); atan(X[2], X[1])]
    
    Args:
        X: State tensor of shape (N, 4)
        Y: Observation tensor of shape (N, 2)
    
    Returns:
        Residual tensor of shape (N, 2)
    """
    Y_pred = observation_function(X)
    return Y - Y_pred

def likelihood(V):
    """Compute likelihood P(Y | X) using Gaussian likelihood
    Julia: fcn_probabilityOF_Y_conditioningON_X(V) = exp(-0.5 * V' * inv(R) * V)
    
    Args:
        V: Residual tensor of shape (N, 2)
    
    Returns:
        Likelihood tensor of shape (N,)
    """
    # For each particle i: exp(-0.5 * V[i]^T * R^{-1} * V[i])
    R_inv = tf.linalg.inv(R)
    # Compute V @ R_inv for all particles: (N, 2) @ (2, 2) = (N, 2)
    temp = tf.matmul(V, R_inv)
    # Compute V^T * (R_inv * V) element-wise and sum: (N, 2) * (N, 2) -> sum -> (N,)
    exponent = -0.5 * tf.reduce_sum(temp * V, axis=1)
    return tf.exp(exponent)

print("Functions defined")

In [ ]:
def random_resample(weights):
    """Random resampling algorithm
    Julia: fcn_random_Resample(vect_weights)
    
    The Julia implementation:
    1. Compute CDF of weights
    2. Generate N uniform random numbers
    3. For each random number, find the first CDF bin it falls into
    
    Args:
        weights: particle weights of shape (N,)
    
    Returns:
        indices: indices of particles to duplicate, shape (N,)
    """
    N = tf.shape(weights)[0]
    
    # Compute CDF: cdf_of_weight = cumsum(vect_weights)
    cdf = tf.cumsum(weights)
    
    # Generate random uniform samples: vect_U = rand(uni_dist, length_of_weight)
    u = tf.random.uniform((N,), 0.0, 1.0, dtype=tf.float64)
    
    # For each u, find the first CDF value >= u
    # This matches the Julia loop logic
    indices = tf.searchsorted(cdf, u, side='right')
    
    return indices

print("Resampling function defined")

In [ ]:
# Initialize state and particles - exactly as in Julia
# X = [20; 20; -0.2; 0]
X_init = tf.constant([20., 20., -0.2, 0.], dtype=tf.float64)

# covMat_original_state = [1.0 0 0 0; 0 1.0 0 0; 0 0 0.01 0; 0 0 0 0.01]
P_init = tf.constant([
    [1.0, 0., 0., 0.],
    [0., 1.0, 0., 0.],
    [0., 0., 0.01, 0.],
    [0., 0., 0., 0.01]
], dtype=tf.float64)

# Number of particles: N = 100
N = 100

# Sample initial particles: pos_particles = rand(state_dist, N)
initial_dist = tfd.MultivariateNormalFullCovariance(
    loc=X_init,
    covariance_matrix=P_init
)

particles = initial_dist.sample(N)  # Shape: (N, 4)
# weights_particles = ones(Float64, N) * (1.0 / N)
weights = tf.ones(N, dtype=tf.float64) / tf.cast(N, tf.float64)

# Storage for results
# X_estimation[1, :] = X and X_prediction[1, :] = X
X_estimation_list = [X_init]
X_prediction_list = [X_init]

print(f"Initialized {N} particles")
print(f"Initial state: {X_init.numpy()}")
print(f"Initial particle mean: {tf.reduce_mean(particles, axis=0).numpy()}")
print(f"Number of observations: {num_steps}")

In [ ]:
# Particle Filter main loop - matching Julia implementation exactly
# for i = 1:size(Y_observation, 1)

for i in range(num_steps):
    # 1. Importance Sampling - Propagate particles
    # state_noise_particles = rand(state_noise_dist, N)
    state_noise = state_noise_dist.sample(N)  # Shape: (N, 2)
    
    # for j = 1:N
    #     pos_particles[:, j] = fcn_X_pre(pos_particles[:, j], state_noise_particles[:, j])
    # end
    particles = state_transition(particles, state_noise)
    
    # 2. Calculate prediction
    # X_pre = mean(pos_particles, dims=2)
    X_pred = tf.reduce_mean(particles, axis=0)
    X_prediction_list.append(X_pred)
    
    # 3. Update weights using observation
    # for j = 1:N
    #     v = fcn_V_between_pre_and_ob(pos_particles[:, j], Y_observation[i, :])
    #     weights_particles[j] = weights_particles[j] * fcn_probabilityOF_Y_conditioningON_X(v)
    # end
    Y_current = tf.tile(Y_observations[i:i+1, :], [N, 1])  # Broadcast observation to (N, 2)
    V = observation_residual(particles, Y_current)
    likelihood_values = likelihood(V)
    
    weights = weights * likelihood_values
    
    # 4. Normalize weights
    # sum_of_weights = sum(weights_particles)
    # weights_particles[j] = weights_particles[j] / sum_of_weights
    weight_sum = tf.reduce_sum(weights)
    if weight_sum < 1e-10:
        print(f"Warning: weights sum is nearly zero at iteration {i}")
        weights = tf.ones(N, dtype=tf.float64) / tf.cast(N, tf.float64)
    else:
        weights = weights / weight_sum
    
    # 5. Resampling
    # duplicat_id = fcn_random_Resample(weights_particles)
    # pos_particles = pos_particles[:, duplicat_id]
    indices = random_resample(weights)
    particles = tf.gather(particles, indices)
    
    # 6. Reset weights after resampling
    # weights_particles = ones(Float64, N) * (1.0 / N)
    weights = tf.ones(N, dtype=tf.float64) / tf.cast(N, tf.float64)
    
    # 7. Calculate estimation
    # X_est = mean(pos_particles, dims=2)
    X_est = tf.reduce_mean(particles, axis=0)
    X_estimation_list.append(X_est)

# Stack results into tensors
X_estimation = tf.stack(X_estimation_list)  # Shape: (num_steps+1, 4)
X_prediction = tf.stack(X_prediction_list)  # Shape: (num_steps+1, 4)

print("\nParticle filter completed")
print(f"Estimation tensor shape: {X_estimation.shape}")
print(f"Prediction tensor shape: {X_prediction.shape}")
print(f"\nFinal estimated state: {X_estimation[-1].numpy()}")
print(f"Final predicted state: {X_prediction[-1].numpy()}")

In [ ]:
# Compute observed positions for plotting
# Julia: df.pos_x = @. df.Range * cos(df.Theta)
#        df.pos_y = @. df.Range * sin(df.Theta)
obs_pos_x = Range * tf.cos(Theta)
obs_pos_y = Range * tf.sin(Theta)

# Julia: pos_x = vcat(X[1], df.pos_x)
#        pos_y = vcat(X[2], df.pos_y)
obs_x_with_init = tf.concat([[X_init[0]], obs_pos_x], axis=0)
obs_y_with_init = tf.concat([[X_init[1]], obs_pos_y], axis=0)

# Create visualization matching Julia plot
plt.figure(figsize=(12, 8))

# Plot observations (blue, dashed line with triangles)
plt.plot(obs_x_with_init.numpy(), obs_y_with_init.numpy(), '--o', 
         color='blue', label='Observation', markersize=6, alpha=0.6)

# Plot predictions (green, dashed line with triangles)
plt.plot(X_prediction[:, 0].numpy(), X_prediction[:, 1].numpy(), '--^', 
         color='green', label='Prediction', markersize=6, alpha=0.6)

# Plot estimations (red, solid line with squares)
plt.plot(X_estimation[:, 0].numpy(), X_estimation[:, 1].numpy(), '-s', 
         color='red', label='Estimation', markersize=6, alpha=0.8)

plt.xlabel('Position X (m)', fontsize=12)
plt.ylabel('Position Y (m)', fontsize=12)
plt.title('Particle Filter Results', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('pf_results.png', dpi=150)
plt.show()

print("Visualization complete")

In [ ]:
# Compute estimation error statistics
obs_positions = tf.stack([obs_pos_x, obs_pos_y], axis=1)  # Shape: (num_steps, 2)
est_positions = X_estimation[1:, :2]  # Shape: (num_steps, 2), skip initial state
pred_positions = X_prediction[1:, :2]  # Shape: (num_steps, 2)

# Compute position errors
estimation_errors = tf.norm(est_positions - obs_positions, axis=1)
prediction_errors = tf.norm(pred_positions - obs_positions, axis=1)

print("\n=== Estimation Error Statistics ===")
print(f"Mean error: {tf.reduce_mean(estimation_errors).numpy():.4f} m")
print(f"Min error: {tf.reduce_min(estimation_errors).numpy():.4f} m")
print(f"Max error: {tf.reduce_max(estimation_errors).numpy():.4f} m")
print(f"Std error: {tfp.stats.stddev(estimation_errors).numpy():.4f} m")

print("\n=== Prediction Error Statistics ===")
print(f"Mean error: {tf.reduce_mean(prediction_errors).numpy():.4f} m")
print(f"Min error: {tf.reduce_min(prediction_errors).numpy():.4f} m")
print(f"Max error: {tf.reduce_max(prediction_errors).numpy():.4f} m")
print(f"Std error: {tfp.stats.stddev(prediction_errors).numpy():.4f} m")

print(f"\nFinal estimated state (x, y, vx, vy): [{X_estimation[-1, 0].numpy():.3f}, {X_estimation[-1, 1].numpy():.3f}, {X_estimation[-1, 2].numpy():.3f}, {X_estimation[-1, 3].numpy():.3f}]")